In [1]:
import pandas as pd

import time

In [2]:
url_qid_query='https://drive.google.com/uc?id=1GIa7RQOTXqvvwFFckfTVVNrtTYvlQwzj'
url_qid_url_rating='https://drive.google.com/uc?id=1kONAJPwnwGhuI7meaHKxEg0P7IDeeKkD' 
url_hostid_url='https://drive.google.com/uc?id=1iw3hNptjKkWHqErjU-pkoRl6ZBnaA0Ke' 

In [3]:
qid_query = pd.read_csv(url_qid_query, sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv(url_qid_url_rating, sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv(url_hostid_url, sep="\t", names=["hostid", "url"])

# делаем join двух таблиц, чтобы было просто брать url с максимальным рейтингом
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")

## pFound c циклом от Yandex Cup

In [4]:
def plook(ind, rels):
  if ind == 0:
    return 1

  return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)

In [5]:
def pfound(group):
    
  max_by_host = group.groupby("hostid")["rating"].max() # максимальный рейтинг хоста

  top10 = max_by_host.sort_values(ascending=False)[:10] # берем топ10 урлов с наивысшим рейтингом
  
  pfound = 0
  for ind, val in enumerate(top10):
    pfound += val*plook(ind, top10.values)
   
  return pfound

In [6]:
start_time = time.time()

qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound) # группируем по qid и вычисляем pfound
qid_max = qid_pfound.idxmax() # берем qid с максимальным pfound

print("Time: ", time.time() - start_time)

print("Mean pFound: ", qid_pfound.mean()) 

Time:  0.0940392017364502
Mean pFound:  0.4603173929969001


## pFound с pandas

In [7]:
def pFound_pandas(df, k, pBreak):
    # df - DataFrame(qid, url, rating, hostid), k - количество документов, pBreak - вероятность остановки просмотра
    
    # максимальный рейтинг host для каждого qid
    df_max_raiting = df.groupby(['qid', 'hostid'])["rating"].max().reset_index() 
    
    # берет топ k элментов по максимальному rating
    df_top_k = df_max_raiting.sort_values(['qid', 'rating'], ascending=False).groupby('qid').head(k)

    # нумерация hostid для каждого qid
    df_top_k['index'] = df_top_k.groupby('qid').cumcount() + 1
    
    # рассчитываем (1 - pRel[i - 1])*(1 - pBreak) 
    df_top_k['mult'] = ((1 - df_top_k['rating']).shift(periods=1)) * (1 - pBreak)
    
    # для каждого hostid с индексом 1 произведение равно 1
    df_top_k.loc[df_top_k['index'] == 1, 'mult'] = 1
    
    # рассчитываем pLook
    df_top_k['pLook'] = df_top_k.groupby('qid')['mult'].cumprod()
    
    # рассчитываем pFound
    df_top_k['pFound'] = df_top_k['pLook'] * df_top_k['rating']

    #возвращаем pFound mean
    return df_top_k.groupby('qid')['pFound'].sum().mean()


In [8]:
start_time = time.time()
mean = pFound_pandas(qid_url_rating_hostid, 10, 0.15)
print("Time: ", time.time() - start_time)

print("Mean pFound: ", mean) 

Time:  0.05469512939453125
Mean pFound:  0.4603173929969002
